<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
#Start Here
import numpy as np
import gensim
import os
import re
from ast import literal_eval

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import tqdm

In [2]:
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [3]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [4]:
df['reviews.text'][0]

'I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.'

In [5]:
def tokenize(text):
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = tokens.lower().split()
    
    return tokens

In [6]:
df['tokens'] = df['reviews.text'].apply(tokenize)

In [7]:
df['tokens']

0        [i, order, 3, of, them, and, one, of, the, ite...
1        [bulk, is, always, the, less, expensive, way, ...
2        [well, they, are, not, duracell, but, for, the...
3        [seem, to, work, as, well, as, name, brand, ba...
4        [these, batteries, are, very, long, lasting, t...
                               ...                        
28327    [i, got, 2, of, these, for, my, 8, yr, old, tw...
28328    [i, bought, this, for, my, niece, for, a, chri...
28329    [very, nice, for, light, internet, browsing, k...
28330    [this, tablet, does, absolutely, everything, i...
28331    [at, ninety, dollars, the, expectionations, ar...
Name: tokens, Length: 28332, dtype: object

In [47]:
stop_words = ['batteries','I', 'amazon', 'i', 'Amazon', 'it', "it's", 'it.', 'the', 'this', 'tablet', 'ive', 'dont', 'brand', 'brands']

import spacy
nlp = spacy.load('en_core_web_lg')

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
stops = nlp.Defaults.stop_words.union(stop_words)

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
def remove_stops(_list_):
    
    keywords = []
    
    for keyword in _list_:
        
        phrase = []
        
        words = keyword.split()
        
        for word in words:
            if word in stops:
                pass
            else: 
                phrase.append(word)
            
        phrase = ' '.join(phrase)
        
        if len(phrase) > 0:
            keywords.append(phrase)
    
    return keywords
    

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
from tqdm import tqdm

tqdm.pandas()

df['clean_reviews'] = df['tokens'].progress_apply(remove_stops)

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 28332/28332 [00:00<00:00, 67738.57it/s]


In [51]:
df['clean_reviews']

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        [order, 3, item, bad, quality, missing, backup...
1                   [bulk, expensive, way, products, like]
2                                 [duracell, price, happy]
3                                    [work, better, price]
4                            [long, lasting, price, great]
                               ...                        
28327    [got, 2, 8, yr, old, twins, 11, yr, old, bette...
28328    [bought, niece, christmas, giftshe, 9, years, ...
28329    [nice, light, internet, browsing, keeping, ema...
28330    [absolutely, want, watch, tv, shows, movies, c...
28331    [ninety, dollars, expectionations, low, good, ...
Name: clean_reviews, Length: 28332, dtype: object

In [52]:
import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
id2word = corpora.Dictionary(df['clean_reviews'])

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
id2word.filter_extremes(no_below=15, no_above=0.85)

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
len(id2word.keys())

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1927

In [56]:
corpus = [id2word.doc2bow(text) for text in df['clean_reviews']]

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
corpus[812]

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(20, 1),
 (25, 1),
 (40, 1),
 (49, 1),
 (92, 1),
 (162, 1),
 (291, 1),
 (311, 1),
 (378, 1),
 (381, 1),
 (399, 1),
 (580, 1),
 (613, 1),
 (816, 1)]

In [58]:
id2word[77]

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'started'

In [59]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=20,
                   passes=50, 
                   workers=12
                  )

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
lda.print_topics()

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.248*"easy" + 0.242*"use" + 0.038*"great" + 0.035*"set" + 0.026*"love" + 0.020*"navigate" + 0.013*"apps" + 0.012*"setup" + 0.011*"simple" + 0.011*"size"'),
 (1,
  '0.040*"light" + 0.033*"screen" + 0.033*"kindle" + 0.028*"size" + 0.021*"great" + 0.021*"reading" + 0.020*"like" + 0.019*"read" + 0.018*"love" + 0.017*"perfect"'),
 (2,
  '0.177*"good" + 0.122*"great" + 0.116*"price" + 0.041*"quality" + 0.033*"battery" + 0.024*"life" + 0.023*"product" + 0.020*"works" + 0.013*"nice" + 0.012*"reading"'),
 (3,
  '0.132*"kids" + 0.035*"parental" + 0.033*"love" + 0.028*"case" + 0.028*"apps" + 0.026*"controls" + 0.022*"great" + 0.017*"like" + 0.015*"control" + 0.014*"set"'),
 (4,
  '0.071*"books" + 0.059*"read" + 0.029*"use" + 0.028*"reading" + 0.021*"music" + 0.020*"kindle" + 0.018*"love" + 0.016*"great" + 0.014*"book" + 0.013*"alexa"'),
 (5,
  '0.102*"games" + 0.044*"play" + 0.036*"love" + 0.030*"kids" + 0.030*"time" + 0.029*"friendly" + 0.028*"kid" + 0.024*"user" + 0.023*"free" + 0.023*

In [61]:
import re

words = [re.findall(r'"[^"]*"', t[1]) for t in lda.print_topics(20)]

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
topic_labels = [', '.join(t[0:5]) for t in words]

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
for id, t in enumerate(topic_labels): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
"easy", "use", "great", "set", "love"

------ Topic 1 ------
"light", "screen", "kindle", "size", "great"

------ Topic 2 ------
"good", "great", "price", "quality", "battery"

------ Topic 3 ------
"kids", "parental", "love", "case", "apps"

------ Topic 4 ------
"books", "read", "use", "reading", "music"

------ Topic 5 ------
"games", "play", "love", "kids", "time"

------ Topic 6 ------
"loves", "bought", "daughter", "gift", "son"

------ Topic 7 ------
"work", "works", "great", "got", "needed"

------ Topic 8 ------
"charge", "kindle", "charging", "time", "new"

------ Topic 9 ------
"fire", "kindle", "hd", "love", "8"

------ Topic 10 ------
"like", "box", "need", "work", "run"

------ Topic 11 ------
"apps", "store", "google", "device", "play"

------ Topic 12 ------
"great", "kids", "lots", "keeps", "features"

------ Topic 13 ------
"great", "product", "recommend", "nice", "highly"

------ Topic 14 ------
"cant", "beat", "price", "like", "know"

------ To

In [64]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/home/ivancampos/.local/share/virtualenvs/DS-Unit-4-Sprint-1-NLP-J60lCYGq/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.125714  0.116332       1        1  10.372583
15     0.246883 -0.129611       2        1   6.591448
4     -0.045738  0.138348       3        1   6.482576
2      0.213804 -0.154271       4        1   6.458358
10     0.113014  0.104924       5        1   6.182770
1      0.020082  0.155393       6        1   5.665835
9     -0.048624  0.150625       7        1   5.367930
11     0.018036  0.194371       8        1   4.873227
16    -0.283450 -0.085169       9        1   4.775201
6     -0.232874 -0.185876      10        1   4.529668
3     -0.140032  0.018761      11        1   4.502927
8     -0.031302  0.140755      12        1   4.443300
5     -0.221775 -0.032082      13        1   4.381922
17     0.153172 -0.057292      14        1   4.175755
18     0.039757 -0.000232      15        1   4.016697
0     -0.039695  0.037145      16        1   3.872435
13     0.118011 -0.181967      17        1   3.864550
7     -0.005347 -0.130084      18        1   3.762676
12    -0.059623 -0.125887      19        1   2.936246
14     0.059988  0.025817      20        1   2.743894, topic_info=           Term         Freq        Total Category  logprob  loglift
660        easy  2958.000000  2958.000000  Default  30.0000  30.0000
84          use  4390.000000  4390.000000  Default  29.0000  29.0000
30         good  5126.000000  5126.000000  Default  28.0000  28.0000
470         old  2129.000000  2129.000000  Default  27.0000  27.0000
25          buy  2170.000000  2170.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1066       turn    44.383531   203.770074  Topic20  -5.1549   2.0717
1240   internet    53.101468   526.383113  Topic20  -4.9756   1.3020
160        nice    56.777780  1217.273362  Topic20  -4.9086   0.5306
10    expensive    49.592796   479.160848  Topic20  -5.0439   1.3276
18        great    63.693375  8768.116343  Topic20  -4.7937  -1.3290

[1209 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
367       1  0.180838        1
367       5  0.139738        1
367       8  0.106859        1
367       9  0.378116        1
367      11  0.004110        1
...     ...       ...      ...
1497     20  0.323456  youtube
1316      9  0.922538       yr
1316     11  0.070014       yr
1000      9  0.810655      yrs
1000     10  0.147392      yrs

[3725 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 16, 5, 3, 11, 2, 10, 12, 17, 7, 4, 9, 6, 18, 19, 1, 14, 8, 13, 15])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling